# Generate Incremental Purchase Behavior Data

Use this notebook to create synthetic behavior updates for existing customers using the same generation logic as the initial setup. It samples a subset of customers, regenerates behavior metrics, and sets UPDATED_AT between each customer's prior UPDATED_AT and the current date.

In [1]:
# Notebook setup
import json
from datetime import datetime

from helper.useful_fns import create_SF_Session, run_sql


/opt/miniconda3/envs/snowflake_ds/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Snowflake session
# Uses defaults from helper.useful_fns.create_SF_Session; adjust as needed.
role, database, schema, session, warehouse = create_SF_Session()

You might have more than one threads sharing the Session object trying to update sql_simplifier_enabled. Updating this while other tasks are running can potentially cause unexpected behavior. Please update the session configuration before starting the threads.



Connection Established with the following parameters:
User                        : JARCHEN
Role                        : "RETAIL_REGRESSION_DEMO_ROLE"
Database                    : "RETAIL_REGRESSION_DEMO"
Schema                      : "DS"
Warehouse                   : "RETAIL_REGRESSION_DEMO_WH"
Snowflake version           : 10.0.0
Snowpark for Python version : 1.38.0 



In [14]:
# Parameters
sample_size = 1000              # Number of customers to refresh; adjust as needed
update_comment = 'Synthetic incremental purchase behavior refresh'
print(f"Updating behavior data for ~{sample_size} randomly selected customers")

Updating behavior data for ~1000 randomly selected customers


In [15]:
# Refresh behavior data for a random subset of existing customers
update_sql = f'''
MERGE INTO PURCHASE_BEHAVIOR pb
USING (
    -- Randomly sample existing customers to refresh
    WITH sampled_customers AS (
        SELECT 
            c.CUSTOMER_ID,
            c.TENURE_MONTHS,
            c.LOYALTY_TIER,
            c.ANNUAL_INCOME,
            COALESCE(pb.UPDATED_AT, DATEADD('day', -90, CURRENT_TIMESTAMP())) AS PREV_UPDATED_AT
        FROM CUSTOMERS c
        JOIN PURCHASE_BEHAVIOR pb ON pb.CUSTOMER_ID = c.CUSTOMER_ID
        QUALIFY ROW_NUMBER() OVER (ORDER BY RANDOM()) <= {sample_size}
    ),
    -- Base metrics (same logic as initial load)
    base_metrics AS (
        SELECT 
            bc.CUSTOMER_ID,
            bc.TENURE_MONTHS,
            bc.LOYALTY_TIER,
            bc.ANNUAL_INCOME,
            CASE 
                WHEN bc.LOYALTY_TIER = 'low' THEN 1
                WHEN bc.LOYALTY_TIER = 'medium' THEN 2
                WHEN bc.LOYALTY_TIER = 'high' THEN 3
                ELSE 1
            END AS TIER_SCORE,
            ROUND(GREATEST(15, LEAST(500,
                UNIFORM(50, 150, RANDOM()) + (bc.ANNUAL_INCOME / 5000) + (CASE WHEN bc.LOYALTY_TIER = 'high' THEN 20 ELSE 0 END) + UNIFORM(-30, 30, RANDOM())
            )), 2) AS AVG_ORDER_VALUE,
            ROUND(GREATEST(0.1, LEAST(8,
                0.5 + (CASE 
                    WHEN bc.LOYALTY_TIER = 'low' THEN 1 * 0.8
                    WHEN bc.LOYALTY_TIER = 'medium' THEN 2 * 0.8
                    WHEN bc.LOYALTY_TIER = 'high' THEN 3 * 0.8
                    ELSE 1 * 0.8 END)
                + (UNIFORM(0, 200, RANDOM()) / 100.0)
                - (CASE WHEN bc.TENURE_MONTHS < 6 THEN 0.3 ELSE 0 END)
            )), 2) AS PURCHASE_FREQUENCY
        FROM sampled_customers bc
    ),
    -- Derived metrics (dependent on base metrics)
    derived_metrics AS (
        SELECT 
            bm.CUSTOMER_ID,
            bm.AVG_ORDER_VALUE,
            bm.PURCHASE_FREQUENCY,
            ROUND(GREATEST(0, LEAST(30,
                UNIFORM(2, 15, RANDOM()) + (CASE WHEN bm.PURCHASE_FREQUENCY > 4 THEN 5 ELSE 0 END) + UNIFORM(-5, 5, RANDOM())
            )), 2) AS RETURN_RATE,
            DATEADD('day', -GREATEST(1, ROUND(30 / GREATEST(bm.PURCHASE_FREQUENCY, 0.5) + UNIFORM(0, 14, RANDOM()))), CURRENT_DATE()) AS LAST_PURCHASE_DATE,
            GREATEST(1, ROUND(bm.PURCHASE_FREQUENCY * bm.TENURE_MONTHS)) AS TOTAL_ORDERS,
            bm.TENURE_MONTHS,
            bm.TIER_SCORE,
            bm.ANNUAL_INCOME
        FROM base_metrics bm
    ),
    -- Final calculations + new UPDATED_AT between previous update and now
    final_calculations AS (
        SELECT
            dm.CUSTOMER_ID,
            dm.AVG_ORDER_VALUE,
            dm.PURCHASE_FREQUENCY,
            dm.RETURN_RATE,
            ROUND(GREATEST(50,
                (dm.AVG_ORDER_VALUE * dm.PURCHASE_FREQUENCY * dm.TENURE_MONTHS)
                * (1 - dm.RETURN_RATE / 100) * (1 + dm.TIER_SCORE * 0.1) * (1 + dm.ANNUAL_INCOME / 500000)
                * (1 + UNIFORM(-0.15, 0.15, RANDOM()))
                * (CASE WHEN UNIFORM(0, 100, RANDOM()) < 4 THEN 0.7 ELSE 1 END)
                * (CASE WHEN UNIFORM(0, 100, RANDOM()) < 4 THEN 1.3 ELSE 1 END)
                + UNIFORM(-800, 800, RANDOM())
            ), 2) AS LIFETIME_VALUE,
            dm.LAST_PURCHASE_DATE,
            dm.TOTAL_ORDERS,
            dm.TENURE_MONTHS,
            dm.TIER_SCORE,
            dm.ANNUAL_INCOME,
            sc.PREV_UPDATED_AT
        FROM derived_metrics dm
        JOIN sampled_customers sc ON sc.CUSTOMER_ID = dm.CUSTOMER_ID
    ),
    updated_behavior AS (
        SELECT 
            CUSTOMER_ID,
            AVG_ORDER_VALUE,
            PURCHASE_FREQUENCY,
            RETURN_RATE,
            LIFETIME_VALUE,
            LAST_PURCHASE_DATE,
            TOTAL_ORDERS,
            -- UPDATED_AT uniformly between previous update and now (only forward in time)
            LEAST(
                DATEADD(
                    'second',
                    ROUND(
                        GREATEST(0, DATEDIFF('second', PREV_UPDATED_AT, CURRENT_TIMESTAMP()))
                        * UNIFORM(0, 1, RANDOM())
                    ),
                    PREV_UPDATED_AT
                ),
                CURRENT_TIMESTAMP()
            ) AS UPDATED_AT
        FROM final_calculations
    )
    SELECT * FROM updated_behavior
) src
ON pb.CUSTOMER_ID = src.CUSTOMER_ID
WHEN MATCHED THEN UPDATE SET
    AVG_ORDER_VALUE = src.AVG_ORDER_VALUE,
    PURCHASE_FREQUENCY = src.PURCHASE_FREQUENCY,
    RETURN_RATE = src.RETURN_RATE,
    LIFETIME_VALUE = src.LIFETIME_VALUE,
    LAST_PURCHASE_DATE = src.LAST_PURCHASE_DATE,
    TOTAL_ORDERS = src.TOTAL_ORDERS,
    UPDATED_AT = src.UPDATED_AT;
'''

# Execute and show merge summary
result = session.sql(update_sql).collect()
print(result)

[Row(number of rows updated=1000)]


In [6]:
# Quick validation: most recent behavior rows
session.sql('''
SELECT *
FROM PURCHASE_BEHAVIOR
ORDER BY UPDATED_AT DESC
LIMIT 10
''').show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"AVG_ORDER_VALUE"  |"PURCHASE_FREQUENCY"  |"RETURN_RATE"  |"LIFETIME_VALUE"  |"LAST_PURCHASE_DATE"  |"TOTAL_ORDERS"  |"UPDATED_AT"                |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------
|138            |99.21              |2.38                  |18.00          |11593.11          |2025-12-27            |119             |2026-01-19 19:13:24.753000  |
|116            |94.23              |4.04                  |5.00           |11517.79          |2026-01-05            |97              |2026-01-19 19:13:24.753000  |
|35             |81.16              |3.45                  |14.00          |6444.60           |2026-01-01            |66              |2026-01-19 19:13:24.753000  |
|222      

In [12]:
# Cleanup: remove behavior rows with UPDATED_AT outside 2025-2026
cleanup_sql = """
DELETE FROM PURCHASE_BEHAVIOR
WHERE YEAR(UPDATED_AT) < 2025
   OR YEAR(UPDATED_AT) > 2026;
"""

rows_deleted = session.sql(cleanup_sql).collect()
print(rows_deleted)


[Row(number of rows deleted=0)]
